In [1]:
import onnx
import tvm
from tvm import relay
import numpy as np
from tvm.contrib import graph_executor

# **載入 ONNX 模型**
onnx_model_path = "/home/11126110/FPGA/量化模型/prun0.75_static_quantized_model.onnx"
onnx_model = onnx.load(onnx_model_path)

# **確保 ONNX 模型形狀正確**
onnx_model = onnx.shape_inference.infer_shapes(onnx_model)

# **定義輸入形狀**
input_name = "images"
input_shape = (1, 3, 640, 640)
shape_dict = {input_name: input_shape}

# **轉換 ONNX 到 Relay IR**
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

# **儲存 Relay IR**
relay_ir_path = "/home/11126110/FPGA/relay_ir.txt"
with open(relay_ir_path, "w") as f:
    f.write(str(mod))
print(f"✅ Relay IR 已儲存至: {relay_ir_path}")


✅ Relay IR 已儲存至: /home/11126110/FPGA/relay_ir.txt


In [3]:
target = tvm.target.Target("llvm -mtriple=aarch64-linux-gnu")
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)


# **儲存編譯後的 FPGA 模型**
compiled_model_path = "/home/11126110/FPGA/compiled_model.tar"
lib.export_library(compiled_model_path)
print(f"✅ TVM FPGA 編譯完成，模型儲存於: {compiled_model_path}")


One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.
pool is not optimized for arm cpu.


✅ TVM FPGA 編譯完成，模型儲存於: /home/11126110/FPGA/compiled_model.tar
